# Metric-based Quantum Architecture Search

## Required imports

In [ ]:
import numpy as np
import scipy

import time
import os

from itertools import combinations, product
from functools import reduce
import copy

import pennylane as qml
from pennylane.templates.layers import RandomLayers
# from pennylane import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt

from sklearn.datasets import make_moons

from scipy.stats import spearmanr

# import gym

In [138]:
!pip install torch torchquantum

  Using cached torch-1.13.0-cp37-cp37m-manylinux1_x86_64.whl (890.2 MB)
  Using cached torchquantum-0.1.4-py3-none-any.whl (119 kB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached torchvision-0.14.0-cp37-cp37m-manylinux1_x86_64.whl (24.3 MB)
  Using cached torchpack-0.3.1-py3-none-any.whl (34 kB)
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached qiskit-0.32.1-py3-none-any.whl
  Using cached pylatexenc-2.10-py3-none-any.whl
  Using cached qiskit_ignis-0.6.0-py3-none-any.whl (207 kB)
  Using cached qiskit_ibmq_provider-0.18.1-py3-none-any.whl (237 kB)
  Using ca

In [2]:
!pip install pennylane pennylane-qiskit qiskit matplotlib

  Using cached PennyLane-0.25.1-py3-none-any.whl (1.0 MB)
  Using cached PennyLane_qiskit-0.24.0-py3-none-any.whl (34 kB)
  Using cached qiskit-0.38.0-py3-none-any.whl
  Using cached matplotlib-3.5.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached retworkx-0.11.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
  Using cached networkx-2.6.3-py3-none-any.whl (1.9 MB)
  Using cached autoray-0.3.2-py3-none-any.whl (36 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached autograd-1.4-py3-none-any.whl (48 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached PennyLane_Lightning-0.25.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.6 MB)
  Using cached ninja-1.10.2.3-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl (108 kB)
  Using cached mthree-1.0.0-cp37-cp37m-manylinux

In [66]:
!pip install torch torchquantum

  Using cached torch-1.12.1-cp37-cp37m-manylinux1_x86_64.whl (776.3 MB)
  Using cached torchquantum-0.1.0-py2.py3-none-any.whl (33 kB)
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
  Using cached torchpack-0.3.1-py3-none-any.whl (34 kB)
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached torchvision-0.13.1-cp37-cp37m-manylinux1_x86_64.whl (19.1 MB)
  Using cached multimethod-1.8-py3-none-any.whl (9.8 kB)
  Using cached loguru-0.6.0-py3-none-any.whl (58 kB)
  Using cached tensorpack-0.11-py2.py3-none-any.whl (296 kB)
  Using cached msgpack_numpy-0.4.8-py2.py3-none-any.whl (6.9 kB)
  Using cached msgpack-1.0.4-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (299 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are i

## 1-RDM separation

In [21]:
def compute_reduced_similarity(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, params, data, meas_qubits=[0]):
    circ = create_batched_gate_circ(qml.device('lightning.qubit', wires=num_qubits), circ_gates, gate_params, inputs_bounds,
                                        weights_bounds, meas_qubits, 'matrix') 

    num_data = len(data)
    traces = []
    circ_fids = np.zeros((num_data, num_data))

    circ_dms = circ(data, params)
    
    for i in range(num_data):
        traces.append(np.trace(circ_dms[i]))

    for s1 in range(num_data):
        for s2 in range(s1 + 1, num_data):
            trace_1 = traces[s1]
            trace_2 = traces[s2]
            fid_trace = np.trace(np.matmul(circ_dms[s1], circ_dms[s2]))

            curr_score = ((fid_trace) ** 2 / (trace_1 * trace_2)).real

            circ_fids[s1, s2] = curr_score
            circ_fids[s2, s1] = curr_score  
            
    circ_fids += np.eye(num_data)
            
    return circ_fids

In [ ]:
!python supernet/train_search_mnist.py --warmup_epochs 0 --steps 25000 --n_search 1 --n_qubits 4 --n_experts 1 --n_layers 1 --n_encode_layers 4 --save "4_params" --data "./experiment_data/mnist_2/" --save_dir "./supernet/mnist_2"

2022-09-23 05:01:10.989900: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-09-23 05:01:10.990005: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2022-09-23 05:01:11.016255: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
(1600, 4, 4) (400, 4, 4)
subnet: [151462], expert_idx: 0
2022-09-23 05:01:22.051044: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-23 05:01:22.076130: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2999995000 Hz
2022-09-23 05:

In [ ]:
!python supernet/train_search_mnist.py --warmup_epochs 0 --steps 25000 --n_search 1 --n_qubits 4 --n_experts 1 --n_layers 2 --n_encode_layers 4 --save "8_params" --data "./experiment_data/mnist_2/" --save_dir "./supernet/mnist_2"

## ranndom circuits

In [ ]:
from datasets import load_dataset
from train_circ import mse_vec_loss

x_train, y_train, x_test, y_test = load_dataset('bank', 'angle', 1)

num_params = [4, 8, 12, 16, 20]

num_qubits = 4
num_embeds = 4

dev = qml.device('lightning.qubit', wires=num_qubits)

for curr_num_params in num_params:
    curr_dir = './random/bank/{}_params'.format(curr_num_params)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
        
    for i in range(25):
        curr_circ_dir = curr_dir + '/circ_{}'.format(i + 1)
        
        if not os.path.exists(curr_circ_dir):
            os.mkdir(curr_circ_dir)
        
        ent_prob = np.random.sample()
        cxz = np.random.sample()
        pauli = 0

        circ_gates, gate_params, inputs_bounds, weights_bounds = generate_true_random_gate_circ(num_qubits, num_embeds, curr_num_params, 
                                                                                                        ent_prob=ent_prob, 
                                                                                                        cxz_prob=cxz * ent_prob,
                                                                                                        pauli_prob=pauli * (
                                                                                                            1 - cxz) * ent_prob)
        
        np.savetxt(curr_circ_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_circ_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_circ_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_circ_dir + '/weights_bounds.txt', weights_bounds)
        
        circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                        weights_bounds, [0], 'exp', 'adjoint') 

        losses_list = []
        accs_list = []
        
        for j in range(5):
            curr_train_dir = curr_circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)
                        
            info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], 5490, 0.05, 1, mse_vec_loss, verbosity=17300, 
                                                                                            loss_window=50, init_params=None, 
                                                                                            acc_thres=1.1, shuffle=True, print_loss=50)

            val_exps = np.array([circ(x_test[i], info[-1][-1]) for i in range(len(x_test))])
            val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(val_loss < 1)

            np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
            np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

            losses_list.append(val_loss)
            accs_list.append(acc)

        np.savetxt(curr_circ_dir + '/accs.txt', accs_list)
        np.savetxt(curr_circ_dir + '/val_losses.txt', losses_list)   

Step 1 | Sliding Loss Window : 1.9999999999999998
Step 51 | Sliding Loss Window : 0.84
Step 101 | Sliding Loss Window : 0.96
Step 151 | Sliding Loss Window : 1.0
Step 201 | Sliding Loss Window : 1.08
Step 251 | Sliding Loss Window : 1.24
Step 301 | Sliding Loss Window : 0.72
Step 351 | Sliding Loss Window : 0.88
Step 401 | Sliding Loss Window : 0.96
Step 451 | Sliding Loss Window : 1.0
Step 501 | Sliding Loss Window : 0.76
Step 551 | Sliding Loss Window : 1.12
Step 601 | Sliding Loss Window : 0.68
Step 651 | Sliding Loss Window : 1.2
Step 701 | Sliding Loss Window : 0.92
Step 751 | Sliding Loss Window : 1.2
Step 801 | Sliding Loss Window : 1.04
Step 851 | Sliding Loss Window : 1.04
Step 901 | Sliding Loss Window : 1.16
Step 951 | Sliding Loss Window : 1.2
Step 1001 | Sliding Loss Window : 0.96
Step 1051 | Sliding Loss Window : 0.92
Step 1101 | Sliding Loss Window : 1.12
Step 1151 | Sliding Loss Window : 0.8
Step 1201 | Sliding Loss Window : 1.0
Step 1251 | Sliding Loss Window : 1.04
St

## human design circs

In [ ]:
from datasets import load_dataset
from create_human_design_circs import generate_human_design_circ

dataset = 'moons_300'
num_reps = 2

num_qubits = 2
dev = qml.device('lightning.qubit', wires=num_qubits)
meas_qubits = [0]

ours_dir = './human_design/amp_basic/moons/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', num_reps)
x_train = x_train.reshape((len(x_train), 1, x_train.shape[1]))
x_test = x_test.reshape((len(x_test), 1, x_test.shape[1]))

num_params = [2 * i for i in range(1, 9)]
params_per_layer = 2

num_embed_layers = 1

dev = qml.device('lightning.qubit', wires=num_qubits)

var_layer_options = [[qml.RY], [qml.RX]] * 4
enc_layer_options = [[None, True]]

for curr_num_params in num_params:
    curr_dir = ours_dir + '{}_params'.format(curr_num_params)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
        
    curr_num_layers = curr_num_params // params_per_layer
    curr_weights_shape = (curr_num_layers, 1, num_qubits)
        
    for i in range(1):
        curr_circ_dir = curr_dir

        circ = generate_human_design_circ(dev, num_qubits, 'amp', 'basic', num_embed_layers, curr_num_layers, enc_layer_options, var_layer_options[:curr_num_layers], [0], 'exp')
        
        losses_list = []
        accs_list = []
        
        for j in range(5):
            curr_train_dir = curr_circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)
                        
            info = train_qnn(circ, x_train, y_train, x_test, y_test, curr_weights_shape, 2400, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                            loss_window=50, init_params=None, 
                                                                                            acc_thres=1.1, shuffle=True, print_loss=50)

            val_exps = np.array([circ(x_test[i], info[-1][-1]) for i in range(len(x_test))])
            val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(val_loss < 1)

            shape = np.array(info[-1]).shape
            
            np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), np.array(info[-1]).reshape(shape[0], np.product(shape[1:])))
            np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

            losses_list.append(val_loss)
            accs_list.append(acc)

        np.savetxt(curr_circ_dir + '/accs.txt', accs_list)
        np.savetxt(curr_circ_dir + '/val_losses.txt', losses_list) 

Step 1 | Sliding Loss Window : 3.0588197186315202
Step 51 | Sliding Loss Window : 1.079412809106036
Step 101 | Sliding Loss Window : 1.0744722567510052
Step 151 | Sliding Loss Window : 1.0677947494489604
Step 201 | Sliding Loss Window : 1.0088584282385293
Step 251 | Sliding Loss Window : 1.099225500444243
Step 301 | Sliding Loss Window : 1.0816312374563983
Step 351 | Sliding Loss Window : 1.0863032230881258
Step 401 | Sliding Loss Window : 1.078474401461576
Step 451 | Sliding Loss Window : 1.0012389682431304
Step 501 | Sliding Loss Window : 1.1031971630910262
Step 551 | Sliding Loss Window : 1.0790290740519268
Step 601 | Sliding Loss Window : 1.0619237020170198
Step 651 | Sliding Loss Window : 1.0720120985717785
Step 701 | Sliding Loss Window : 1.0476675839475285
Step 751 | Sliding Loss Window : 1.0600579865558188
Step 801 | Sliding Loss Window : 1.083496804578453
Step 851 | Sliding Loss Window : 1.09967027736677
Step 901 | Sliding Loss Window : 1.069039158393921
Step 951 | Sliding Los

## suprennt circuits

In [37]:
def convert_supernet_circ_into_gate_circ(subnet, num_embeds, layer_rots, layer_cnots, num_qubits, angle_embed=False):
    circ_gates = []
    gate_params = []
    weights_bounds = [0]
    inputs_bounds = [0]

    curr_layers = subnet
    num_cnots = len(layer_cnots)
    
    for i in range(1):
        num_qubits = len(layer_rots[0])

        if angle_embed:
            rots = ['ry', 'rx', 'rz']
            
            for j in range(num_embeds):
                circ_gates += [rots[j % 3] for i in range(num_qubits)]
                gate_params += [[i] for i in range(num_qubits)]
                weights_bounds += [0 for i in range(num_qubits)]
                inputs_bounds += [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]            
        else:
            for j in range(num_embeds):
                circ_gates += ['h' for i in range(num_qubits)] + ['ry' for i in range(num_qubits)] + ['cry' for i in range(num_qubits - 1)]
                gate_params += [[i] for i in range(num_qubits)] * 2 + [[i, i + 1] for i in range(num_qubits - 1)]
                weights_bounds += [0 for i in range(3 * num_qubits - 1)]
                inputs_bounds += [inputs_bounds[-1] for i in range(num_qubits)] + [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]

        for j in range(len(curr_layers)):
            circ_gates += layer_rots[curr_layers[j] // num_cnots]
            circ_gates += ['cx' for k in layer_cnots[curr_layers[j] % num_cnots]]

            gate_params += [[k] for k in range(len(layer_rots[curr_layers[j] // num_cnots]))]
            gate_params += layer_cnots[curr_layers[j] % num_cnots]

            weights_bounds += [weights_bounds[-1] + k + 1 for k in range(num_qubits)]
            inputs_bounds += [inputs_bounds[-1] for k in range(num_qubits)]

            weights_bounds += [weights_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            inputs_bounds += [inputs_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            
    return circ_gates, gate_params, inputs_bounds, weights_bounds

In [ ]:
import pennylane as qml
import numpy as np
import os

from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ
from create_noise_models import noisy_dev_from_backend

dataset = 'mnist_2'
main_dir = './supernet/mnist_2/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 1)

num_qubits = 4
num_cnot_configs = 4096
num_circs = 2500
num_embed_layers = 4

device_name = 'ibmq_lima'
# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

param_nums = [8, 12, 16, 20]

for p in param_nums:
    supernet_dir = main_dir + 'search-{}_params_mb'.format(p)

    num_layers = p // num_qubits

    layer_rots = open(supernet_dir + '/rotations.txt').read().split('\n')[:-1]
    layer_rots = [''.join([j for j in i if j.isupper()]) for i in layer_rots]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = open(supernet_dir + '/cnots.txt').read().split('\n')[:-1]
    layer_cnots = [''.join([j for j in i[1:] if j not in ['[', ']', ',', ' ']]) for i in layer_cnots]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]
        
    search_space = len(layer_rots) * len(layer_cnots)
        
    accs = []
    losses = []
    circ_layers = []

    params = np.genfromtxt(supernet_dir + '/training_params.txt')[-1].reshape((num_layers, search_space // num_cnot_configs, num_qubits))
        
    supernet_device_dir = supernet_dir + '/' + device_name
    
    if not os.path.exists(supernet_device_dir):
        os.mkdir(supernet_device_dir)
        
    for i in range(num_circs):
        curr_circ_desc = np.random.randint(0, search_space, num_layers)
#         curr_circ_desc = [[272920], [163863], [258091], [172861], [126492]][i]

        curr_params = np.concatenate([params[k, curr_circ_desc[k] // num_cnot_configs] for k in range(num_layers)]).flatten()

        circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(curr_circ_desc, num_embed_layers, layer_rots, layer_cnots,
                                                                                                          num_qubits, False) 
        
        circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, [0], 'exp')
        
        val_exp_list = []
        
        for j in range(len(x_test)):
            val_exp_list.append(circ(x_test[j], curr_params))
            
        val_exps = np.array(val_exp_list).flatten()
        val_losses = np.power(val_exps - y_test, 2)
        val_loss = np.mean(val_losses)
        acc = np.mean(np.multiply(val_exps, y_test) > 0)
            
        print(curr_circ_desc, acc, val_loss)

        accs.append(acc)
        losses.append(val_loss)
        circ_layers.append(curr_circ_desc)
        
    np.savetxt(supernet_device_dir + '/searched_circ_layers.txt', np.array(circ_layers))
    np.savetxt(supernet_device_dir + '/searched_circ_accs.txt', accs)
    np.savetxt(supernet_device_dir + '/searched_circ_losses.txt', losses)  
#     np.savetxt(supernet_dir + '/searched_circ_accs.txt', ['{} {}'.format(circ_layers[i], accs[i]) for i in range(num_circs)], fmt="%s")

ibmqfactory.load_account:WARNING:2022-10-02 14:00:27,441: Credentials are already in use. The existing account in the session will be replaced.


[22007 13888] 0.545 1.0312944316864014
[313629 219683] 0.5025 1.1062126159667969
[108118 172841] 0.49 1.0693354988098145
[129815 182309] 0.4925 1.0710063648223878
[165383 182834] 0.5 1.0595513916015624
[291634 299755] 0.5225 1.0270887565612794
[303740  61252] 0.4875 1.0382840061187744
[147308 269710] 0.4825 1.1894860363006592
[106918 192941] 0.4975 1.1269544219970704
[152205 192100] 0.5175 1.0294874572753907
[71775 92834] 0.4625 1.049727725982666
[ 18065 162096] 0.4725 1.0946965885162354
[138051 316763] 0.5325 1.0468227195739745
[242647 182768] 0.4875 1.0120526790618896
[ 33028 236133] 0.5225 1.038199815750122
[270918 268885] 0.5225 1.0171918964385986
[290134  38533] 0.545 1.0063713741302491
[189965 165818] 0.5125 1.0728435230255127
[ 26412 266388] 0.5125 1.0921786975860597
[216357 134095] 0.485 1.026086778640747
[258350 159413] 0.4625 1.0286151123046876
[190492  58380] 0.5 1.0561509704589844
[268668   8199] 0.47 1.0112923049926759
[68699 42665] 0.48 1.0920231819152832
[102055 253000] 

In [10]:
print(supernet_dir)
print(inputs_bounds[-1])

./supernet/bank/search-4_params_mb
14


## our metrc

In [ ]:
from datasets import load_dataset
from create_gate_circs import create_batched_gate_circ
from metrics import compute_reduced_similarity

search_params = [4, 8, 12, 16, 20]

ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16))))
ideal = np.concatenate((ideal, ideal[::-1, :]), 1)

num_qubits = 4
num_embeds = 16
meas_qubits = [0]

x_train, y_train, x_test, y_test = load_dataset('mnist_2', 'angle', 1)

class_0_sel = np.random.choice(800, 16, False)
class_1_sel = np.random.choice(800, 16, False) + 800
sel_inds = np.concatenate((class_0_sel, class_1_sel))

sel_data = x_train[sel_inds]

# np.savetxt('./ours/mnist_2/sel_data.txt', sel_data)

dev = qml.device('lightning.qubit', wires=num_qubits)

for p in search_params:
    curr_dir = './ours/mnist_2/{}_params'.format(p)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
    
    d2_min_scores = []
    d2_mean_scores = []
    d2_var_scores = []
    
    d2_t_min_scores = []
    d2_t_mean_scores = []
    d2_t_var_scores = []
    
    mean_mat_scores = []
    mean_t_mat_scores = []

    for i in range(2500):
        curr_circ_dir = curr_dir + '/circ_{}'.format(i + 1)

        if not os.path.exists(curr_circ_dir):
            os.mkdir(curr_circ_dir)
        
        ent_prob = np.random.sample()
        cxz = np.random.sample()
        pauli = 0

        params = 2 * np.pi * np.random.sample((32, p))
#         np.savetxt(curr_circ_dir + '/params.txt', params)
        
        circ_gates, gate_params, inputs_bounds, weights_bounds = generate_true_random_gate_circ(num_qubits, num_embeds, p, 
                                                                                                        ent_prob=ent_prob, 
                                                                                                        cxz_prob=cxz * ent_prob,
                                                                                                        pauli_prob=pauli * (
                                                                                                            1 - cxz) * ent_prob)
        
#         np.savetxt(curr_circ_dir + '/gates.txt', circ_gates, fmt="%s")
#         np.savetxt(curr_circ_dir + '/gate_params.txt', gate_params, fmt="%s")
#         np.savetxt(curr_circ_dir + '/inputs_bounds.txt', inputs_bounds)
#         np.savetxt(curr_circ_dir + '/weights_bounds.txt', weights_bounds)

        batched_circ = create_batched_gate_circ(qml.device('lightning.qubit', wires=num_qubits), circ_gates, gate_params, inputs_bounds,
                                            weights_bounds, meas_qubits, 'matrix') 

        circ_d2_scores = []
        circ_d2_t_scores = []
        circ_mean_mat = np.zeros((32, 32))
        circ_t_mean_mat = np.zeros((32, 32))

        for j in range(32):
            curr_params = np.concatenate([params[j] for k in range(32)]).reshape((32, p))
            mat = compute_reduced_similarity(batched_circ, curr_params, sel_data)
            
            t_mat = mat > ((np.sum(mat) - 32) / 996)
        
            diff_mat = mat - ideal
            diff_2d = np.sum(np.multiply(diff_mat, diff_mat))

            diff_t_mat = t_mat - ideal
            diff_2d_t = np.sum(np.multiply(diff_t_mat, diff_t_mat))

            circ_d2_scores.append(diff_2d)
            circ_d2_t_scores.append(diff_2d_t)

            circ_mean_mat += mat / 32
            circ_t_mean_mat += t_mat / 32
            
#         np.savetxt(curr_circ_dir + '/d2_scores.txt', circ_d2_scores)
#         np.savetxt(curr_circ_dir + '/d2_t_scores.txt', circ_d2_t_scores)
        
        d2_min_scores.append(np.min(circ_d2_scores))
        d2_mean_scores.append(np.mean(circ_d2_scores))
        d2_var_scores.append(np.var(circ_d2_scores))
        
        d2_t_min_scores.append(np.min(circ_d2_t_scores))
        d2_t_mean_scores.append(np.mean(circ_d2_t_scores))
        d2_t_var_scores.append(np.var(circ_d2_t_scores))
        
        diff_mean_mat = ideal - circ_mean_mat
        diff_t_mean_mat = ideal - circ_t_mean_mat
        
        mean_mat_scores.append(np.sum(np.multiply(diff_mean_mat, diff_mean_mat)))
        mean_t_mat_scores.append(np.sum(np.multiply(diff_t_mean_mat, diff_t_mean_mat)))
        
        print(i)
        
#     np.savetxt(curr_dir + '/d2_mean_scores.txt', d2_mean_scores)
#     np.savetxt(curr_dir + '/d2_min_scores.txt', d2_min_scores)
#     np.savetxt(curr_dir + '/d2_var_scores.txt', d2_var_scores)
    
#     np.savetxt(curr_dir + '/d2_t_mean_scores.txt', d2_t_mean_scores)
#     np.savetxt(curr_dir + '/d2_t_min_scores.txt', d2_t_min_scores)
#     np.savetxt(curr_dir + '/d2_t_var_scores.txt', d2_t_var_scores)
    
#     np.savetxt(curr_dir + '/d2_mean_mat_scores.txt', mean_mat_scores)
#     np.savetxt(curr_dir + '/d2_mean_t_mat_scores.txt', mean_t_mat_scores)

## train our circuits

In [ ]:
from datasets_nt import load_dataset
from create_gate_circs import get_circ_params, create_gate_circ
from train_circ import train_qnn, mse_loss
from create_noise_models import noisy_dev_from_backend

search_nums = [100]
param_nums = [4, 8, 12, 16, 20]

dataset = 'bank'
num_reps = 2

num_qubits = 4
dev = qml.device('lightning.qubit', wires=num_qubits)
meas_qubits = [0]
mat_size = 1024

device_name = 'ibm_nairobi'
noisy_dev = noisy_dev_from_backend(device_name, num_qubits)

ours_dir = './ours/bank/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', num_reps)

for search_num in search_nums:       
    for param_num in param_nums:
        noise_scores = []
        
        for i in range(2500):
            noise_scores.append(np.genfromtxt(ours_dir + '/{}_params/circ_{}/noise_metric/{}/metric_tvd_score.txt'.format(param_num, i + 1, device_name)))
            
        noise_scores = np.array(noise_scores)
        
        mean_t_mat_scores = np.genfromtxt(ours_dir + '{}_params/d2_mean_t_mat_scores.txt'.format(param_num))
        perf_scores = (mat_size - mean_t_mat_scores) / mat_size
        combined_scores = np.multiply(perf_scores, noise_scores)
        
        circ_inds = np.random.permutation(2500)
        
        curr_dir = ours_dir + '/{}_params/search_{}_{}'.format(param_num, search_num, device_name)
        
        if not os.path.exists(curr_dir):
            os.mkdir(curr_dir)

        for j in range(25):
            curr_trial_dir = curr_dir + '/trial_{}'.format(j + 1)

            if not os.path.exists(curr_trial_dir):
                os.mkdir(curr_trial_dir)
        
            sel_inds = circ_inds[range(j * 100, j * 100 + 100)]
            sel_ind = sel_inds[np.argmax(combined_scores[sel_inds])]

            np.savetxt(curr_trial_dir + '/searched_circuit_inds.txt', sel_inds)
            np.savetxt(curr_trial_dir + '/searched_circuit_scores.txt', combined_scores[sel_inds])
            np.savetxt(curr_trial_dir + '/sel_circuit_ind.txt', [sel_ind])
            np.savetxt(curr_trial_dir + '/sel_circuit_score.txt', [combined_scores[sel_ind]])
            
            print(sel_inds, combined_scores[sel_inds])
            print(sel_ind, combined_scores[sel_ind])
            print(noise_scores[sel_ind], perf_scores[sel_ind])

            circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(ours_dir + '{}_params/circ_{}'.format(param_num, sel_ind + 1))

            circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', 'adjoint')
            
            noisy_circ = create_gate_circ(noisy_dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', None)
        
            losses_list = []
            accs_list = []
            noisy_losses_list = []
            noisy_accs_list = []

            for j in range(5):
                curr_train_dir = curr_trial_dir + '/run_{}'.format(j + 1)

                if os.path.exists(curr_train_dir):
                    pass
                else:
                    os.mkdir(curr_train_dir)


                info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], 5490, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                                loss_window=50, init_params=None, 
                                                                                                acc_thres=1.1, shuffle=True, print_loss=50)

                val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()
                
                noisy_val_exps = [noisy_circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                noisy_val_loss = np.array([mse_loss(y_test[k], noisy_val_exps[k]) for k in range(len(x_test))]).flatten()

                acc = np.mean(val_loss < 1)
                noisy_acc = np.mean(noisy_val_loss < 1)
#                 acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)

                np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
                np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])             

                losses_list.append(val_loss)
                accs_list.append(acc)
                
                noisy_losses_list.append(noisy_val_loss)
                noisy_accs_list.append(noisy_acc)

            np.savetxt(curr_trial_dir + '/accs.txt', accs_list)
            np.savetxt(curr_trial_dir + '/val_losses.txt', losses_list)          
            np.savetxt(curr_trial_dir + '/noisy_accs.txt', noisy_accs_list)
            np.savetxt(curr_trial_dir + '/noisy_val_losses.txt', noisy_losses_list) 

ibmqfactory.load_account:WARNING:2022-09-26 17:58:52,757: Credentials are already in use. The existing account in the session will be replaced.


[1067 2082 1279 2370 1200   57 1131  720 1096  134  991  175 2133  455
 1992 1644  768 1804 1429 1133 2064   92 1102  919 2063  663 1039 2121
   27 2153 2243  179 1034  533 1877 2346  890 1832 2367 1553  825  268
  128  671 1619  918 1800  628 1610  146  217  100 1582  104 1595 1327
 2221 1374 1969 1990  827 2291 1360  599 2324  892 2093  467  578 1298
 1681 2347 1742 2487  659 1095 1323 2014  759   19  526 1577 2494  630
 2354  190 1618 1693   34 1662  243  376 2234 1591  562 2010  147  622
 2254 1286] [0.35647452 0.44221437 0.46940085 0.50470418 0.49201727 0.4377847
 0.57586987 0.47526027 0.42752922 0.45651078 0.54055554 0.45351791
 0.57267149 0.56381992 0.62488468 0.45341074 0.47422796 0.55102779
 0.56935706 0.56489674 0.44816101 0.61210878 0.44867516 0.5373931
 0.43963069 0.58558509 0.60000924 0.6152313  0.64009358 0.4437561
 0.57632821 0.58982131 0.55132762 0.59124615 0.615347   0.55868626
 0.58827229 0.50437095 0.59651261 0.49384058 0.4331398  0.5064426
 0.59331018 0.4532665  0.5

## correlaation circuits

In [ ]:
from create_noise_models import noisy_dev_from_backend
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ, get_circ_params
from train_circ import train_qnn, mse_loss

import numpy as np
import os

dataset = 'mnist_2'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 2)

num_qubits = 4
num_embeds = 32
num_params = 12

device_name = 'ibm_nairobi'

# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

for i in range(600, 800):
    curr_dir = './experiment_data/{}/trained_circuits/circ_{}'.format(dataset, i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0], 'exp')
    
    
    noiseless_losses = np.genfromtxt(curr_dir + '/val_losses.txt')

    losses_list = []
    accs_list = []
    
    curr_dev_dir = curr_dir + '/' + device_name

#     if not os.path.exists(curr_dev_dir + '/accs_inference_only.txt'):
    if True:
        if not os.path.exists(curr_dev_dir):
            os.mkdir(curr_dev_dir)

        for j in range(5):
            curr_train_dir = curr_dir + '/run_{}'.format(j + 1)
            curr_params = np.genfromtxt(curr_train_dir + '/params_{}.txt'.format(j + 1))[-1]

            val_exps = [circ(x_test[i], curr_params) for i in range(len(x_test))]
            val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

#             acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
            acc = np.mean(val_loss < 1)

            losses_list.append(val_loss)
            accs_list.append(acc)

        print(np.mean(noiseless_losses), np.mean(losses_list), i + 1)

        np.save(curr_dev_dir + '/val_losses_inference_only.npy', losses_list)
        np.savetxt(curr_dev_dir + '/accs_inference_only.txt', accs_list)
    else:
        print(i)

In [140]:
from importlib import reload

import datasets
import create_gate_circs
import train_circ

reload(datasets)
reload(create_gate_circs)
reload(train_circ)

<module 'train_circ' from '/root/train_circ.py'>

In [ ]:
from datasets import TorchDataset
from create_gate_circs_np import get_circ_params, TQCirc, generate_true_random_gate_circ
from train_circ_np import train_tq_model, TQMseLoss

import pickle as pkl
import torch
import os
import numpy as np

dataset = 'fmnist_2'
curr_dir = f'./experiment_data/{dataset}/trained_circuits/'

num_qubits = 4
num_embeds = 16

num_train_steps = 2500
num_test_data = 400

num_meas_qubits = 1

loss = TQMseLoss(num_meas_qubits, 4)

dev = qml.device('lightning.qubit', wires=4)
device = 'cpu'

train_data = TorchDataset(dataset, 'angle', 1, reshape_labels=True)
test_data = TorchDataset(dataset, 'angle', 1, False, reshape_labels=True)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=32, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=32, sampler=torch.utils.data.SequentialSampler(test_data))

for i in range(800, 1000):    
    circ_dir = curr_dir + f'circ_{i + 1}'
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

    losses_list = []
    accs_list = []
    
    for j in range(5):
        curr_train_dir = circ_dir + '/run_{}'.format(j + 1)
        
        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, False).to(device)
        opt = torch.optim.SGD(model.parameters(), lr=0.05)
    
        curr_loss, curr_acc = train_tq_model(model, num_meas_qubits, opt, loss, train_data_loader, test_data_loader, num_test_data, num_train_steps, 100, 10)
        
        print(curr_loss, curr_acc)
        
        torch.save(model.state_dict(), curr_train_dir + '/model.pt')

        losses_list.append(curr_loss)
        accs_list.append(curr_acc)
        
    np.savetxt(circ_dir + '/val_losses.txt', losses_list)
    np.savetxt(circ_dir + '/accs.txt', accs_list)   

Step 1 | Loss: 1.3369333744049072
Step 101 | Loss: 1.0034929513931274
Step 201 | Loss: 0.9121675491333008
Step 301 | Loss: 0.8650904893875122
Step 401 | Loss: 0.8072453141212463
Step 501 | Loss: 0.7626935839653015
Step 601 | Loss: 0.6002445220947266
Step 701 | Loss: 0.631428599357605
Step 801 | Loss: 0.6582334041595459
Step 901 | Loss: 0.523783266544342
Step 1001 | Loss: 0.6060664653778076
Step 1101 | Loss: 0.6223019957542419
Step 1201 | Loss: 0.534239649772644
Step 1301 | Loss: 0.553539514541626
Step 1401 | Loss: 0.5586219429969788
Step 1501 | Loss: 0.5730311274528503
Step 1601 | Loss: 0.5850419998168945
Step 1701 | Loss: 0.6638661623001099
Step 1801 | Loss: 0.5224451422691345
Step 1901 | Loss: 0.5741580128669739
Step 2001 | Loss: 0.5830962061882019
Step 2101 | Loss: 0.6019691824913025
Step 2201 | Loss: 0.6239913702011108
Step 2301 | Loss: 0.46938976645469666
Step 2401 | Loss: 0.520551323890686
0.5545499068688317 0.8775
Step 1 | Loss: 1.051339864730835
Step 101 | Loss: 0.7728914022445

## noise metris for correlation circuits

In [44]:
from importlib import reload

import create_gate_circs

reload(create_gate_circs)

<module 'create_gate_circs' from '/root/create_gate_circs.py'>

In [ ]:
from create_noise_models import get_real_backend_dev, noisy_dev_from_backend
from create_gate_circs import create_batched_gate_circ, get_circ_params
from datasets_nt import load_dataset
from metrics import compute_noise_metric

dataset = 'fmnist_2'

x_train, y_train, _, __ = load_dataset(dataset, 'angle', 1)

curr_dir = './ours/{}'.format(dataset)

num_qubits = 4
num_trial_params = 128
meas_qubits = [0, 1, 2, 3]

param_nums = [16, 20, 24]

num_cdcs = 128
num_shots = 1024

device_name = 'ibm_oslo'

dev = noisy_dev_from_backend(device_name, num_qubits)
noiseless_dev = qml.device('lightning.qubit', wires=num_qubits)

score_tvds = []
actual_tvds = []

for p in param_nums:
    param_dir = curr_dir + '/{}_params'.format(p)
    
    for i in range(2500):
        circ_dir = param_dir + '/circ_{}'.format(i + 1)

        circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

        noisy_circ = create_batched_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                    weights_bounds, meas_qubits, 'probs') 

        noiseless_circ = create_batched_gate_circ(noiseless_dev, circ_gates, gate_params, inputs_bounds,
                                                                    weights_bounds, meas_qubits, 'probs') 

        noise_metric_dir = circ_dir + '/noise_metric'

        if not os.path.exists(noise_metric_dir):
            os.mkdir(noise_metric_dir)

        device_noise_metric_dir = noise_metric_dir + '/{}'.format(device_name)

        if not os.path.exists(device_noise_metric_dir):
            os.mkdir(device_noise_metric_dir)

        params = np.random.sample((num_trial_params, weights_bounds[-1])) * 2 * np.pi
        batch_data = x_train[np.random.choice(len(x_train), num_trial_params, False)]

        noiseless_res_raw = np.array(noiseless_circ(batch_data, params, shots=num_shots))
        noisy_res_raw = np.array(noisy_circ(batch_data, params, shots=num_shots))

        actual_tvd = 1 - np.mean(np.sum(0.5 * np.abs(noiseless_res_raw - noisy_res_raw), 1))

        tvd = compute_noise_metric(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, noiseless_dev, dev, num_cdcs=num_cdcs, num_shots=num_shots)

        np.savetxt(device_noise_metric_dir + '/metric_tvd_score.txt', [1 - tvd])
        np.savetxt(device_noise_metric_dir + '/actual_tvd_score.txt', [actual_tvd])

        score_tvds.append(1 - tvd)
        actual_tvds.append(actual_tvd)

        print(1 - tvd, actual_tvd, i)
        
    np.savetxt(param_dir + '/act_tvds.txt', actual_tvds)
    np.savetxt(param_dir + '/metric_tvds.txt', score_tvds)

ibmqfactory.load_account:WARNING:2022-11-08 09:36:00,149: Credentials are already in use. The existing account in the session will be replaced.


0.8274917602539062 0.8373260498046875 0
0.9281463623046875 0.923980712890625 1
0.8760147094726562 0.8889083862304688 2
0.49957275390625 0.6451644897460938 3
0.57659912109375 0.6991958618164062 4
0.7619247436523438 0.7932968139648438 5
0.703277587890625 0.70416259765625 6
0.8485794067382812 0.8330307006835938 7
0.5754165649414062 0.6112213134765625 8
0.430877685546875 0.21771240234375 9
0.9415969848632812 0.9387130737304688 10
0.6963424682617188 0.7333602905273438 11
0.7458953857421875 0.7218856811523438 12
0.6496124267578125 0.6650619506835938 13
0.5886001586914062 0.6567916870117188 14
0.7606353759765625 0.786041259765625 15
0.8507156372070312 0.8256378173828125 16
0.9256973266601562 0.7718124389648438 17
0.5761260986328125 0.6028823852539062 18
0.6667938232421875 0.7109451293945312 19
0.7703323364257812 0.7227630615234375 20
0.7348251342773438 0.7983856201171875 21
0.7387771606445312 0.7050247192382812 22
0.7100830078125 0.717620849609375 23
0.41106414794921875 0.596649169921875 24
0

In [ ]:
from create_noise_models import noisy_dev_from_backend
from datasets import load_dataset
from create_gate_circs import create_gate_circ, get_circ_params
from train_circ import train_qnn, mse_vec_loss

import numpy as np
import os

dataset = 'fmnist_4'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 1)

num_qubits = 4
num_embeds = 16
num_params = 18

device_name = 'ibmq_belem'

# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

for i in range(300, 400):
    curr_dir = './experiment_data/fmnist_4/trained_circuits/circ_{}'.format(i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0, 1], 'exp')
    
    
    noiseless_losses = np.genfromtxt(curr_dir + '/val_losses.txt')

    losses_list = []
    accs_list = []
    
    curr_dev_dir = curr_dir + '/' + device_name

    if not os.path.exists(curr_dev_dir + '/accs_inference_only.txt'):   
        if not os.path.exists(curr_dev_dir):
            os.mkdir(curr_dev_dir)

        for j in range(5):
            curr_train_dir = curr_dir + '/run_{}'.format(j + 1)
            curr_params = np.genfromtxt(curr_train_dir + '/params_{}.txt'.format(j + 1))[-1]

            val_exps = [circ(x_test[i], curr_params) for i in range(len(x_test))]
            val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
    #         acc = np.mean(val_loss < 1)

            losses_list.append(val_loss)
            accs_list.append(acc)

        print(np.mean(noiseless_losses), np.mean(losses_list), i + 1)

        np.save(curr_dev_dir + '/val_losses_inference_only.npy', losses_list)
        np.savetxt(curr_dev_dir + '/accs_inference_only.txt', accs_list)
    else:
        print(i)

ibmqfactory.load_account:WARNING:2022-09-14 04:19:31,785: Credentials are already in use. The existing account in the session will be replaced.
/usr/local/lib/python3.7/site-packages/qiskit/providers/aer/noise/device/models.py:266: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (92038.51932346408 > 2 * 2551.431037973455). Truncating to maximum value.
  " Truncating to maximum value.", UserWarning)
/usr/local/lib/python3.7/site-packages/qiskit/providers/aer/noise/device/models.py:266: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (92038.51932346408 > 2 * 2551.431037973455). Truncating to maximum value.
  " Truncating to maximum value.", UserWarning)
/usr/local/lib/python3.7/site-packages/qiskit/providers/aer/noise/device/models.py:266: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T

1.199634404683853 1.21474975512387 301
1.0374430730151272 1.239637414514575 302
1.066567561734048 1.1227739556105196 303
1.3398419045390846 1.3422871892537744 304
1.7071067811865477 1.6404855063268897 305
0.890458522069639 0.9055996595060093 306
1.2506872330476575 1.2626274584599355 307
1.1278161449880393 1.1617028852908189 308
1.0739668298766045 1.0855949210758071 309


In [ ]:
from create_noise_models import get_real_backend_dev, noisy_dev_from_backend
from create_gate_circs import create_batched_gate_circ, get_circ_params
from datasets_nt import load_dataset
from metrics import compute_noise_metric

dataset = 'vowel_4'

x_train, y_train, _, __ = load_dataset(dataset, 'angle', 1)

curr_dir = './experiment_data/{}/trained_circuits'.format(dataset)

num_qubits = 4
num_trial_params = 128
meas_qubits = [0, 1, 2, 3]

num_cdcs = 128
num_shots = 1024

device_name = 'ibm_oslo'

dev = noisy_dev_from_backend(device_name, num_qubits)
noiseless_dev = qml.device('lightning.qubit', wires=num_qubits)

score_tvds = []
actual_tvds = []

for i in range(1000):
    circ_dir = curr_dir + '/circ_{}'.format(i + 1)
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

    noisy_circ = create_batched_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'probs') 

    noiseless_circ = create_batched_gate_circ(noiseless_dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'probs') 
    
    noise_metric_dir = circ_dir + '/noise_metric'
    
    if not os.path.exists(noise_metric_dir):
        os.mkdir(noise_metric_dir)
    
    device_noise_metric_dir = noise_metric_dir + '/{}'.format(device_name)

    if not os.path.exists(device_noise_metric_dir):
        os.mkdir(device_noise_metric_dir)
    
    params = np.random.sample((num_trial_params, weights_bounds[-1])) * 2 * np.pi
    batch_data = x_train[np.random.choice(len(x_train), num_trial_params, False)]
    
    noiseless_res_raw = np.array(noiseless_circ(batch_data, params, shots=num_shots))
    noisy_res_raw = np.array(noisy_circ(batch_data, params, shots=num_shots))

    actual_tvd = 1 - np.mean(np.sum(0.5 * np.abs(noiseless_res_raw - noisy_res_raw), 1))
    
    tvd = compute_noise_metric(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, noiseless_dev, dev, num_cdcs=num_cdcs, num_shots=num_shots)
    
    np.savetxt(device_noise_metric_dir + '/metric_tvd_score.txt', [1 - tvd])
    np.savetxt(device_noise_metric_dir + '/actual_tvd_score.txt', [actual_tvd])
    
    score_tvds.append(1 - tvd)
    actual_tvds.append(actual_tvd)
    
    print(1 - tvd, actual_tvd, i)

# np.savetxt(curr_dir + '/noise_scores.txt', score_tvds)
# np.savetxt(curr_dir + '/noisy_fids.txt', actual_tvds)

ibmqfactory.load_account:WARNING:2022-11-05 17:24:05,338: Credentials are already in use. The existing account in the session will be replaced.


0.39470672607421875 0.5943145751953125 0
0.945465087890625 0.943634033203125 1
0.5880661010742188 0.6153717041015625 2
0.6478347778320312 0.6675186157226562 3
0.6592636108398438 0.6649246215820312 4
0.4537353515625 0.64727783203125 5
0.39359283447265625 0.5631256103515625 6
0.6658706665039062 0.67681884765625 7
0.5809097290039062 0.6390304565429688 8
0.40633392333984375 0.6287765502929688 9
0.6321182250976562 0.6239395141601562 10
